In [ ]:
import numpy as np
import rasterio as rio
from rasterio.crs import CRS
from affine import Affine
from matplotlib import pyplot
%matplotlib inline
import os

In [ ]:
# How to configure transforms for rasters??

def create_transform(row_width,row_rotation, upper_right_x,
                    column_rotation, column_height, upper_right_y):
    
    return(Affine(row_width,row_rotation,upper_right_x,
                  column_rotation, column_height, upper_right_y))

row_width=0
column_height=0
row_rotation=0
column_rotation=0
upper_right_x=-180
upper_right_y=90

degree1 = create_transform(1,0,-180,0,-1,90)
arcminutes30 = create_transform(.5,0,-180,0,-.5,90)
arcminutes15 = create_transform(.25,0,-180,0,-.25,90)
arcminutes5 = create_transform(0.0833333,0,-180,0,-0.0833333,90)
arcminutes3 = create_transform(row_width,0,-180,0,column_height,90)
arcminutes1 = create_transform(0.0166667,0,-180,0,-0.0166667,90)
arcseconds30 = create_transform(0.00833333,0,-180,-0,0.00833333,90)
arcseconds15 = create_transform(0.00416667,0,-180,-0,0.00416667,90)
arcseconds5 = create_transform(0.00138889,0,-180,0,-0.00138889,90)
arcseconds3 = create_transform(row_width,0,-180,0,column_height,90)
arcseconds1 = create_transform(0.0166667,0,-180,0,-0.0166667,90)

In [ ]:
# Create array of ones size of a global, 1 degree raster
dst_array = np.ones((360, 180), dtype=np.float32)
#dst_array = np.random.randn(360, 180).astype(np.float32)
#print(dst_array)

# Use rasterio to classify this as WGS84 EPSG:4326
world_array = '/Users/nathansuberi/Desktop/RW_Data/world_array.tif'

profile = {
    'driver': 'GTiff', 
    'dtype': np.float32, 
    'nodata': 0, 
    'width': 360, 
    'height': 180, 
    'count': 1, 
    'crs': CRS({'init': 'EPSG:4326'}), 
    'transform':degree1,
    'blockxsize': 128, 
    'blockysize': 128, 
    'tiled': True, 
    'compress': 'lzw', 
    'interleave': 'band'
}
with rio.open(world_array, "w", **profile) as dst:
    dst.write(dst_array, indexes=1)

# Print, observe
with rio.open(world_array, "r") as src:
    data = src.read(indexes=1)
    print(src.profile)
    print(data)
pyplot.imshow(data)

In [ ]:
# Define alternate projection
alt_proj = "EPSG:54009"

In [ ]:
# Use gdal to re-project as Mollweide EPSG:54009
os.environ["Zsrc_file"] = world_array
world_array_edit = world_array[:-4] + "_edit.tif"
os.environ["Zdst_file"] = world_array_edit
os.environ["Zoptions"] = "-r near -s_srs EPSG:4326 -t_srs "+alt_proj+" -of GTiff -overwrite"
!gdalwarp $Zoptions $Zsrc_file $Zdst_file

# Print, observe
with rio.open(world_array_edit, "r") as src:
    data = src.read(indexes=1)
    print(src.profile)
    print(data)
    print(np.mean(data))
    
pyplot.imshow(data)
!gdalinfo $Zdst_file

In [ ]:
# Use gdal to re-project as WGS84 EPSG:4326
os.environ["Zsrc_file"] = world_array_edit
world_array2 = world_array[:-4] + "2.tif"
os.environ["Zdst_file"] = world_array2 

os.environ["Zoptions"] = "-s_srs "+alt_proj+" -t_srs EPSG:4326 -of GTiff -overwrite"
os.environ["Zoptions_with_tr"] = "-r near -s_srs "+alt_proj+" -t_srs EPSG:4326 -of GTiff -overwrite -tr 1 1 -te -180 -90 180 90 -wo SOURCE_EXTRA=1000"
#!gdalwarp $Zoptions $Zsrc_file $Zdst_file
!gdalwarp $Zoptions_with_tr $Zsrc_file $Zdst_file

# Print, observe
with rio.open(world_array2, "r") as src:
    data = src.read(indexes=1)
    print(src.profile)
    print(data)
    print(np.mean(data))
    
pyplot.imshow(data)
!gdalinfo $Zdst_file

In [ ]:
# Use gdal to re-project as WGS84 EPSG:4326
os.environ["Zsrc_file"] = world_array
world_array3 = world_array[:-4] + "3.tif"
os.environ["Zdst_file"] = world_array3
os.environ["Zoptions"] = "-s_srs EPSG:4326 -t_srs EPSG:4326 -of GTiff -overwrite"
!gdalwarp $Zoptions $Zsrc_file $Zdst_file

# Print, observe
with rio.open(world_array3, "r") as src:
    data = src.read(indexes=1)
    print(src.profile)
    print(data)
    print(np.mean(data))
    
pyplot.imshow(data)
!gdalinfo $Zdst_file

In [ ]:
### Create areal weighted rasters
# https://waterprogramming.wordpress.com/2015/06/09/using-arcpy-to-calculate-area-weighted-averages-of-gridded-spatial-data-over-political-units-part-2/
# http://pythonhosted.org/rasterstats/cli.html
# http://mathforum.org/library/drmath/view/63767.html

# https://stackoverflow.com/questions/41826750/calculating-the-area-of-gridded-data-equidistant-in-degrees
# http://unidata.github.io/netcdf4-python/#section1
from netCDF4 import Dataset
import numpy as np
import os
from pyproj import Proj
# Didn't end up using this:
# from shapely.geometry import shape

def calc_cell_area(lon, lon_res, num_cols):
    # TO DO: Bring in Thomas' formula
    R = 6371.

    lon = lon*(np.pi/180)
    lon_res = lon_res*(np.pi/180)

    theta1 = lon
    theta2 = lon-lon_res
    
    h1 = R*np.sin(theta1)
    h2 = R*np.sin(theta2)
    
    A = 2*np.pi*R*np.abs(h1-h2) / num_cols
    
    #print("lon:", lon, ", area:", A)
    return(A)

def create_areal_raster(file_name, start_lat, start_lon, end_lat, end_lon, resolution, col_vector=True):
    # Create netcdf from scratch
    Data = Dataset(file_name, mode="w")

    row_range = (start_lat - end_lat)
    col_range = (end_lon - start_lon)
    
    num_cols = int(col_range / resolution)
    num_rows = int(row_range / resolution)

    Data.createDimension("dim_data_col_vector", num_rows*num_cols)
    #Data.createDimension("dim_data_2d_array", (num_rows, num_cols))

    col_lats = [np.repeat(i, num_rows) for i in np.arange(-180, 180, resolution)]
    row_lons = [np.arange(90, -90, -resolution)]*num_cols
    
    if col_vector:
        lats = Data.createVariable('latitude', 'f4', 'dim_data_col_vector')
        lons = Data.createVariable('longitude', 'f4', 'dim_data_col_vector')
        lats[:] = np.reshape(col_lats, -1)
        lons[:] = np.reshape(row_lons, -1)
        
        # Create a vector of areas for longitude = -180, over whole range of latitudes
        # Broadcast this to be size of AREA_VAR
        AREA_VAR = np.empty(len(lats))
        #projection = Proj(init="EPSG:4326")
        
        areas = []
        
        for j in np.arange(start_lat,end_lat,-resolution):  #just northern hemisphere
            areas.append(calc_cell_area(j,resolution,num_cols))
            
        #print(areas)
        areas = np.repeat(areas, num_cols)
        AREA_VAR = np.reshape(areas, -1)
        #print(AREA_VAR)
        
        cell_area = Data.createVariable('cell_area', 'f4', 'dim_data_col_vector')            
        cell_area[:] = AREA_VAR
        
    else:
        lats = Data.createVariable('latitude', 'f4', 'dim_data_2d_array')
        lons = Data.createVariable('longitude', 'f4', 'dim_data_2d_array')
        # TO DO: Implement 2d array version

    Data.close()
    
    
os.chdir("/Users/nathansuberi/Desktop/RW_Data/Areal_Rasters/")

raster_set = {
    "./1_degree_areal_raster.nc":1,
    "./30_arc_minute_areal_raster.nc":1/2.,
    "./15_arc_minute_areal_raster.nc":1/4.,
    "./5_arc_minute_areal_raster.nc":1/12.,
    "./3_arc_minute_areal_raster.nc":1/20.,
    "./1_arc_minute_areal_raster.nc":1/60.,
    "./30_arc_second_areal_raster.nc":1/120.,
    "./15_arc_second_areal_raster.nc":1/240.,
    "./5_arc_second_areal_raster.nc":1/720.,
    "./3_arc_second_areal_raster.nc":1/1200.,
    "./1_arc_second_areal_raster.nc":1/3600.,   
}

for file_name, resolution in raster_set.items():
    areal_rasters_args = {
        "file_name":file_name, 
        "start_lat": 90, 
        "start_lon": -180, 
        "end_lat": -90, 
        "end_lon": 180, 
        "resolution":resolution, 
        "col_vector":True
        }
    
    try:
        create_areal_raster(**areal_rasters_args)
        print("Done")
    except:
        print("File already exists - permission denied.")

Done
Done
Done
Done
Done
Done
Done


In [ ]:
Read_data = Dataset("./test_areal_raster24.nc", "r+")
Read_data.variables["latitude"]
areas = Read_data.variables["cell_area"][:]
areas[1000:2000]

In [ ]:
# Extract geotiff data, put into a NETCDF4 band


In [ ]:
### Use WFP Logistics Cluster data - create a raster
# 1 = WFP deployment to an area
# 0 = no WFP deployment to an area
# Divide up by month into many rasters, can use as training data labels
# Add in geocoded "Who does what where" from reliefweb
# Try to use GDELT as training data inputs

# Can 1Concern help here?

In [ ]:
# Fitting a gamma distribution to calculation SPI
# Other weighted moving average calculations over raster stacks